In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from math import *
import scipy.stats
from pandas import *
#import scipy.integrate as integrate
from scipy.integrate import quad
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from mpl_toolkits import mplot3d

import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.graph_objects as go



In [221]:
def jump_distribution_full(x_value,mu,sigma_jump,step_x):
    lower = x_value - step_x/2.0
    upper = x_value + step_x/2.0
    #print(lower,upper)
    def normal_distribution_function(x):
        value = scipy.stats.norm.pdf(x,mu,sigma_jump)
        return value
    res, err = quad(normal_distribution_function, lower, upper)
    return res  #output is a vector of the average densities around the regions of each value in the vector x


In [222]:
x0 = 0.0
xn = 10.0
xsteps = 100
dx = (xn-x0)/xsteps
x = np.arange(x0, xn+dx, dx) #last x value is x = 99 
#print(x,len(x),dx)

In [223]:
t0 = 0.0
tn = 1.0
tsteps = 100
dt = (tn-t0)/tsteps
t = np.arange(t0, tn+dt, dt) #last time value is t = 999.9
r = dt / (dx**2) # ensure r < 1
#print(t,r,len(t),tsteps,dt)

In [224]:
def a_coeff_rs(x_value, sigma, kappa, theta, jump_rate,q_12,q_13):
    coefficient = jump_rate + (sigma**2)/(dx**2) - q_12 - q_13
    return coefficient

def b_coeff_rs(x_value, sigma, kappa, theta, jump_rate):
    #print(sigma,kappa,theta,jump_rate)
    coefficient_2 = 0.5*(sigma**2)*(1/dx**2) + 0.5*(1/dx)*kappa*(theta-x_value)
    #print(coefficient_2)
    return coefficient_2

def c_coeff_rs(x_value, sigma, kappa, theta, jump_rate):
    coefficient_3 = 0.5*(sigma**2)*(1/dx**2) - 0.5*(1/dx)*kappa*(theta-x_value)
    return coefficient_3   

In [225]:
#M_1 = M_matrix(x,0.1,0.7,1.0,1.0)
#M_inv = np.linalg.inv(M_1)
#DataFrame(M_1)

In [226]:
def M_matrix_rs(x,sigma,kappa, theta, jump_rate,q_12, q_13):
    M = np.zeros(shape=(len(x),len(x)))
    M[0,0] = 1.0
    M[len(x)-1, len(x)-1] = 1.0
    
    for i in range(1,(len(x)-1)):
        a_coefficient = a_coeff_rs(x[i], sigma, kappa, theta, jump_rate,q_12,q_13)
        b_coefficient = b_coeff_rs(x[i], sigma, kappa, theta, jump_rate)
        c_coefficient = c_coeff_rs(x[i], sigma, kappa, theta, jump_rate)
        if a_coefficient < 0 or b_coefficient < 0 or c_coefficient <0:
            print('Error')
        M[i,(i-1):(i+2)] = [-dt*c_coefficient, 1+dt*a_coefficient, -dt*b_coefficient]
      
    return M

In [242]:
def M_final_rs(x,sigma_1,sigma_2,kappa_1, kappa_2, theta_1, theta_2,jump_rate, q_12, q_13, q_21, q_23):
    M_f = np.zeros(shape=(len(x),len(x)))
    M_block_1 = M_matrix_rs(x,sigma_1,kappa_1,theta_1,jump_rate,q_12, q_13)
    M_block_2 = M_matrix_rs(x,sigma_2,kappa_2,theta_2,jump_rate,q_21, q_23)
    #print(sigma_2,kappa_2,theta_2,jump_rate,q_21,q_23)
    #print(DataFrame(M_block_2))

    X_1 = np.eye(len(x))*(dt*q_12)
    X_1[0] = 0.0
    X_1[len(x)-1] = 0.0 
    X_2 = np.eye(len(x))*(dt*q_13)
    X_2[0] = 0.0
    X_2[len(x)-1] = 0.0 
    Y_1 = np.eye(len(x))*(dt*q_21)
    Y_1[0] = 0.0
    Y_1[len(x)-1] = 0.0
    Y_2 = np.eye(len(x))*(dt*q_23) 
    Y_2[0] = 0.0
    Y_2[len(x)-1] = 0.0 
    M_final_matrix = np.block([
        [M_block_1, X_1, X_2 ],
        [Y_1, M_block_2 , Y_2],
        [np.zeros((len(x),len(x))), np.zeros((len(x),len(x))), np.eye(len(x))] 
    ])
    return M_final_matrix


In [243]:
test = M_final_rs(x,1,1.5,0.5,0.5,0.5,0.2,0.2,q_12=0.1,q_13=0.1, q_21=0.2, q_23=0.2)
DataFrame(test[len(x):2*len(x),len(x):2*len(x)])

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1.0000,0.000,0.0000,0.000,0.0000,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0000,0.000,0.0000,0.000,0.0000
1,-1.1225,3.248,-1.1275,0.000,0.0000,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0000,0.000,0.0000,0.000,0.0000
2,0.0000,-1.125,3.2480,-1.125,0.0000,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0000,0.000,0.0000,0.000,0.0000
3,0.0000,0.000,-1.1275,3.248,-1.1225,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0000,0.000,0.0000,0.000,0.0000
4,0.0000,0.000,0.0000,-1.130,3.2480,-1.12,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0000,0.000,0.0000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0000,0.000,0.0000,0.000,0.0000,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.36,3.2480,-0.890,0.0000,0.000,0.0000
97,0.0000,0.000,0.0000,0.000,0.0000,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,-1.3625,3.248,-0.8875,0.000,0.0000
98,0.0000,0.000,0.0000,0.000,0.0000,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0000,-1.365,3.2480,-0.885,0.0000
99,0.0000,0.000,0.0000,0.000,0.0000,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0000,0.000,-1.3675,3.248,-0.8825


In [244]:
def N_matrix_rs(x, jump_rate, N, mu, sigma_jump, step_x):
    N_m = np.zeros(shape=(len(x),len(x)))
    positions = np.arange(int(-N/2+1),int(N/2),1)
    for i in range(1,(len(x)-1)):
        for j in positions: 
            x_jump = step_x*j
            integral = jump_distribution_full(x_jump, mu, sigma_jump, step_x)
            if (i+j) >= 0 and (i+j) <= (len(x)-1):
                #print(j)
                N_m[i,i+j] = jump_rate*dt*integral
        N_m[i,i] = N_m[i,i]+1
    return N_m

In [245]:
def N_final_rs(x, jump_rate, N,mu,sigma_jump,step_x):
    N_1 = N_matrix_rs(x, jump_rate, N,mu,sigma_jump,step_x)
    N_final_matrix = np.block([
        [N_1, np.zeros((len(x),len(x))), np.zeros((len(x),len(x))) ],
        [np.zeros((len(x),len(x))), N_1, np.zeros((len(x),len(x)))], 
        [np.zeros((len(x),len(x))), np.zeros((len(x),len(x))), np.zeros((len(x),len(x)))]
    ])
    
    return N_final_matrix

In [246]:
N_try = N_final_rs(x, 1.0,4,0.0,0.1,dx)
(N_try[2*len(x):3*len(x),2*len(x):3*len(x)])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [247]:
#N_try = N_matrix(x, 1.0,4,0.0,0.1,dx)
#DataFrame(N_try[0:10, 0:10])

In [248]:
#def IC_matrix(x, sigma, kappa, theta, jump_rate, mu, sigma_jump, step_x, phi_0, phi_N):
#    b = np.zeros(len(x)-2)
#    B = np.zeros(len(x)-2)
#    b_coefficient = b_coeff(x[1], sigma, kappa, theta, jump_rate)
#    c_coefficient = c_coeff(x[len(x)-2], sigma, kappa, theta, jump_rate)
#    b[0] = c_coefficient*dt*phi_0
#    b[len(x)-3] = b_coefficient*dt*phi_N
    #print(b)
#    jump_0 = jump_distribution_full(-1*dx, mu, sigma_jump, step_x)
#    jump_N = jump_distribution_full(1*dx, mu, sigma_jump, step_x)
#    B[0] = jump_rate*dt*jump_0
#    B[len(x)-3] = jump_rate*dt*jump_N
    
#    IC = b+B
#    return IC

In [249]:
#IC_matrix(x,1,1,1,1,0,1,dx,0,1)

In [250]:
b_1 = np.zeros(len(x))
b_1[len(x)-1] = 1.0
b_2 = np.zeros(len(x))
b_2[len(x)-1] = 1.0
b_3 = np.zeros(len(x))
b = np.block([b_1, b_2, b_3]) 

In [251]:
x_block = np.block([x,x,x])
phi_matrix = np.zeros(shape=(3*len(x),len(t))) 
phi_matrix[:,0] = 1.0 * (x_block > 0.0)
phi_matrix[len(x)-1,:] = 1.0
phi_matrix[2*len(x)-1,:] = 1.0
phi_matrix[(2*len(x)):(3*len(x)),:] = 0.0
#DataFrame(phi_matrix)
#phi_matrix = np.zeros(shape=(len(x), len(t)))
#phi_matrix[:,0] = 1.0 * (x>0.0)
#phi_matrix[len(x)-1,:] = 1.0

In [271]:
q_12 = 0.2
q_13 = 0.1
q_21 = 0.4
q_23 = 0.5
sigma_choice_1 = 5.0
kappa_choice_1 = 7.0
theta_choice_1 = 5.0
sigma_choice_2 = 3.0
kappa_choice_2 = 4.0
theta_choice_2 = 10.0
jump_rate_choice = 2.0
N_choice = 4
mu_choice = 0.0
sigma_jump_choice = 5.0

M = M_final_rs(x,sigma_choice_1,sigma_choice_2, kappa_choice_1, kappa_choice_2,
                theta_choice_1, theta_choice_2, jump_rate_choice, q_12, q_13, q_21, q_23)
#print(DataFrame(M[2*len(x):3*len(x),2*len(x):3*len(x)]))
N_mat = N_final_rs(x, jump_rate = jump_rate_choice, N = N_choice, mu = mu_choice, sigma_jump = sigma_jump_choice, step_x=dx)
print(DataFrame(N_mat[2*len(x):3*len(x),2*len(x):3*len(x)]))
if sigma_choice_1**2 >= np.max(np.abs(dx*kappa_choice_1*(theta_choice_1-x))) and sigma_choice_2**2 >= np.max(np.abs(dx*kappa_choice_2*(theta_choice_2-x))):
    for time in range(1, len(t)): 
        phi_matrix[:,time] = np.linalg.inv(M).dot(N_mat.dot(phi_matrix[:,time-1]) + b)
else:
    print('Stability conditions not satisfied')
    

     0    1    2    3    4    5    6    7    8    9    ...  91   92   93   \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
96   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
97   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
98   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
99   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
100  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     94   95   96   97   98   99   100  
0    0.0  0.0  0.0  0.0  0.0  0.0 

In [272]:
accept_x = np.where((x>=0))
accept_x[0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int64)

In [273]:
phi_matrix[1,]

array([1.        , 0.35594365, 0.26904401, 0.24150829, 0.22718763,
       0.21765687, 0.21039071, 0.20437752, 0.19913486, 0.19440463,
       0.19003695, 0.18593924, 0.18205143, 0.17833299, 0.1747556 ,
       0.17129889, 0.16794785, 0.16469112, 0.16151995, 0.15842744,
       0.15540804, 0.15245723, 0.14957125, 0.14674694, 0.14398156,
       0.14127275, 0.13861842, 0.13601671, 0.13346595, 0.13096459,
       0.12851123, 0.12610455, 0.12374332, 0.12142639, 0.11915266,
       0.11692108, 0.11473065, 0.11258041, 0.11046944, 0.10839684,
       0.10636175, 0.10436333, 0.10240077, 0.10047327, 0.09858008,
       0.09672045, 0.09489366, 0.09309899, 0.09133577, 0.08960333,
       0.08790101, 0.0862282 , 0.08458427, 0.08296863, 0.08138069,
       0.0798199 , 0.0782857 , 0.07677756, 0.07529496, 0.07383739,
       0.07240436, 0.07099539, 0.06961001, 0.06824778, 0.06690825,
       0.06559099, 0.06429558, 0.06302161, 0.06176868, 0.06053641,
       0.05932443, 0.05813235, 0.05695982, 0.05580649, 0.05467

In [274]:
for i in range(len(x),2*len(x)-1):
    print(np.min(phi_matrix[i+1,] - phi_matrix[i,]))

0.26714108377894374
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-3.137467530855975e-05
-4.9960787642744364e-05
-5.150586468816165e-05
-4.698821075133086e-05
-4.082097009383556e-05
-3.469547035594989e-05
-2.9190523332944807e-05
-2.4439806504794603e-05
-2.0408785869618917e-05
-1.700826174944048e-05
-1.4145675995336227e-05
-1.1774505760397957e-05
-9.773681703517667e-06
-8.075326978662645e-06
-6.637869256920226e-06
-5.461949814100642e-06
-4.452299541157778e-06
-3.594502088866136e-06
-2.904399427405835e-06
-2.3031571049214605e-06
-1.819634845912077e-06
-1.4172102200671333e-06
-1.0811953287115372e-06
-8.208555943856055e-07
-5.995984522222386e-07
-4.383204421642972e-07
-3.036554683255588e-07
-2.089108197544931e-07
-1.358647271354485e-07
-8.360322467648729e-08
-5.058182528383526e-08
-2.673943810815871e-08
-1.3452784441980725e-08
-6.290091669747255e-09
-2.362217754736662e-09
-6.709823807682369e-10
-1.2799217241621363e-10
-1.3311352020650702e-11
-4.04121180963557e-14
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [276]:
def survival(space,time):
    return phi_matrix[space, time]


In [277]:
X = np.arange(0,len(phi_matrix),1)
T = np.arange(0,len(t),1)
X, T = np.meshgrid(X, T)
Phi = survival(X,T)

data=go.Surface(z=Phi, x=x0+dx*X, y=t0+dt*T)

layout = go.Layout(scene = dict(
                    xaxis_title='Initial Position',
                    yaxis_title='Maturity',
                    zaxis_title='Survival Probability'))

fig = go.Figure(data=[data], layout=layout)
pyoff.plot(fig)

'temp-plot.html'

In [1463]:
surv = DataFrame(phi_matrix)

In [31]:
surv['initial_pos'] = x #x0+dx*X[1]

In [32]:
times= DataFrame(t)
time= times[0]

In [33]:
surv.append(time)

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,initial_pos
0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-10.00
1,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.98
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.96
3,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.94
4,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,9.94
998,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,9.96
999,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,9.98
1000,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,10.00


In [34]:
y = np.where((x >= 0.0) & (x<= 1.2))
surv_df_non_zero = surv.iloc[y[0],]
surv_df_non_zero

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,initial_pos
501,1.0,0.456936,0.308519,0.247997,0.215148,0.193969,0.178828,0.167262,0.158013,0.150367,...,0.043889,0.043555,0.043228,0.042907,0.042593,0.042285,0.041984,0.041688,0.041399,0.02
502,1.0,0.702831,0.540755,0.452508,0.398534,0.361864,0.334948,0.314072,0.297214,0.283183,...,0.083230,0.082598,0.081978,0.081371,0.080777,0.080195,0.079624,0.079065,0.078517,0.04
503,1.0,0.835334,0.701578,0.610614,0.547929,0.502502,0.467902,0.440441,0.417924,0.398979,...,0.118594,0.117696,0.116817,0.115955,0.115111,0.114284,0.113473,0.112680,0.111902,0.06
504,1.0,0.906826,0.807603,0.726956,0.664857,0.616669,0.578348,0.547055,0.520886,0.498551,...,0.150475,0.149341,0.148231,0.147143,0.146077,0.145033,0.144010,0.143007,0.142025,0.08
505,1.0,0.945429,0.875271,0.809368,0.753273,0.706698,0.667918,0.635223,0.607246,0.582958,...,0.179300,0.177958,0.176643,0.175355,0.174093,0.172856,0.171644,0.170457,0.169294,0.10
506,1.0,0.966297,0.917483,0.866032,0.818203,0.775860,0.738933,0.706733,0.678483,0.653487,...,0.205442,0.203916,0.202421,0.200956,0.199520,0.198114,0.196737,0.195387,0.194064,0.12
507,1.0,0.977589,0.943376,0.904077,0.864714,0.827762,0.794066,0.763669,0.736298,0.711585,...,0.229226,0.227538,0.225885,0.224265,0.222678,0.221124,0.219600,0.218108,0.216645,0.14
508,1.0,0.983704,0.959053,0.929132,0.897329,0.865910,0.836049,0.808214,0.782489,0.758777,...,0.250937,0.249109,0.247318,0.245564,0.243845,0.242160,0.240510,0.238893,0.237309,0.16
509,1.0,0.987017,0.968446,0.945368,0.919786,0.893434,0.867458,0.842497,0.818845,0.796593,...,0.270828,0.268878,0.266969,0.265097,0.263264,0.261468,0.259708,0.257983,0.256294,0.18
510,1.0,0.988810,0.974024,0.955747,0.935003,0.912964,0.890577,0.868479,0.847054,0.826501,...,0.289120,0.287067,0.285055,0.283085,0.281153,0.279261,0.277407,0.275591,0.273811,0.20


In [35]:
accept_t = [0,10,20,30,40,50,60,70,80,90,100]

In [36]:
times = t[accept_t[1:]]
times

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [37]:
output=surv_df_non_zero.iloc[:,accept_t[1:]]
initial_pos = surv['initial_pos'][(surv['initial_pos'] >= 0.0) & (surv['initial_pos'] <= 1.0)]
output['initial_pos'] = initial_pos
survival_values = round(output,4)
survival_values = survival_values.set_index('initial_pos')
#survival_values.columns = t[accept_t]
#output
survival_values

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.1439,0.1072,0.0887,0.0761,0.0667,0.0592,0.0533,0.0485,0.0446,0.0414
0.04,0.2712,0.2029,0.1679,0.1442,0.1263,0.1123,0.1010,0.0919,0.0845,0.0785
0.06,0.3827,0.2879,0.2387,0.2051,0.1798,0.1598,0.1439,0.1309,0.1204,0.1119
0.08,0.4792,0.3633,0.3018,0.2596,0.2277,0.2025,0.1824,0.1661,0.1528,0.1420
0.10,0.5616,0.4299,0.3581,0.3084,0.2707,0.2409,0.2171,0.1978,0.1821,0.1693
0.12,0.6312,0.4884,0.4081,0.3520,0.3093,0.2755,0.2484,0.2264,0.2086,0.1941
0.14,0.6892,0.5397,0.4526,0.3910,0.3439,0.3066,0.2767,0.2524,0.2327,0.2166
0.16,0.7369,0.5844,0.4920,0.4259,0.3751,0.3348,0.3023,0.2761,0.2547,0.2373
0.18,0.7757,0.6232,0.5269,0.4570,0.4031,0.3602,0.3256,0.2976,0.2748,0.2563


In [39]:
pd = 1-survival_values
pd

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.8561,0.8928,0.9113,0.9239,0.9333,0.9408,0.9467,0.9515,0.9554,0.9586
0.04,0.7288,0.7971,0.8321,0.8558,0.8737,0.8877,0.8990,0.9081,0.9155,0.9215
0.06,0.6173,0.7121,0.7613,0.7949,0.8202,0.8402,0.8561,0.8691,0.8796,0.8881
0.08,0.5208,0.6367,0.6982,0.7404,0.7723,0.7975,0.8176,0.8339,0.8472,0.8580
0.10,0.4384,0.5701,0.6419,0.6916,0.7293,0.7591,0.7829,0.8022,0.8179,0.8307
0.12,0.3688,0.5116,0.5919,0.6480,0.6907,0.7245,0.7516,0.7736,0.7914,0.8059
0.14,0.3108,0.4603,0.5474,0.6090,0.6561,0.6934,0.7233,0.7476,0.7673,0.7834
0.16,0.2631,0.4156,0.5080,0.5741,0.6249,0.6652,0.6977,0.7239,0.7453,0.7627
0.18,0.2243,0.3768,0.4731,0.5430,0.5969,0.6398,0.6744,0.7024,0.7252,0.7437


In [41]:
x = np.array([1.00, 0.4384,0.1931,0.1134,0.0938, 0.0904, 0.0903, 0.0897, 0.0853, 0.0681, 0.00])

In [42]:
x*0.75

array([0.75    , 0.3288  , 0.144825, 0.08505 , 0.07035 , 0.0678  ,
       0.067725, 0.067275, 0.063975, 0.051075, 0.      ])

In [43]:
x = np.array([0.1134,0.2369,0.3365,0.4154,0.4786,0.5297,0.5711,0.6049,0.6323,0.6547])
y = np.array([0,0.1134,0.2369,0.3365,0.4154,0.4786,0.5297,0.5711,0.6049,0.6323])
x-y

array([0.1134, 0.1235, 0.0996, 0.0789, 0.0632, 0.0511, 0.0414, 0.0338,
       0.0274, 0.0224])

In [44]:
pos = np.array([100, 90,80,70,60,50,40,30,20,10])

In [45]:
pos*(x-y)*0.75

array([8.505  , 8.33625, 5.976  , 4.14225, 2.844  , 1.91625, 1.242  ,
       0.7605 , 0.411  , 0.168  ])

In [46]:
sum(pos*(x-y)*0.75)

34.30125

In [36]:
(x-y)*0.75

array([0.0876  , 0.0945  , 0.07485 , 0.058725, 0.0468  , 0.037575,
       0.03045 , 0.024675, 0.0201  , 0.01635 ])